In [1]:
import pandas as pd
import numpy as np
import os,sys,pickle,glob,gc
from tqdm import tqdm

In [ ]:
import findspark
os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3' #'/home/antslab/anaconda3/bin/python'
findspark.init()
import pyspark
# from pyspark import SparkContext
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
from tqdm import tqdm
from pyspark.sql import Row, SQLContext, SparkSession, window
from pyspark import SparkConf, SparkContext
# Import data types
from pyspark.sql.types import *
import  pyspark.sql.functions as F
conf = SparkConf()
conf.set("spark.local.dir", "/mnt/ssd240g/data/Leo_Spark_Home/tmp")
conf.set("spark.executor.cores","12")
conf.set("spark.driver.memory","79g")
conf.set("spark.executor.memory", "89g")
conf.set("spark.memory.offHeap.enabled","true")
conf.set("spark.memory.offHeap.size","75g")
conf.set("spark.driver.extraJavaOptions","-Xss19990m")
conf.set("spark.driver.extraJavaOptions","-Xms79g")
conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
conf.set("spark.sql.execution.arrow.enabled", "true")
conf.set("spark.kubernetes.pyspark.pythonVersion","3")
conf.set("spark.sql.shuffle.partitions",99990)
conf.set("spark.driver.maxResultSize", "20g")
conf.set("spark.sql.debug.maxToStringFields", 190000)
conf.set("spark.sql.hive.filesourcePartitionFileCacheSize",99*1024*1024*1024)
spark = SparkSession.builder.master("local").appName("Test_spark").config(conf=conf).getOrCreate()
sc = SparkContext(conf=conf)
spark.conf.set("spark.sql.execution.arrow.enabled", True)
import databricks.koalas as ks
ks.options.display.max_rows = 20
ks.set_option('compute.max_rows', None)
# ks.set_option('compute.ops_on_diff_frames', True)
ks.set_option('compute.default_index_type', 'distributed')
print(ks.__version__)

In [2]:
# in_parquet = '../../data_hdd4t/pcap_process/pcap_spark/2020_01_09/中華電信/parquet/2020_01_09_country_session.parquet/'
in_parquet = '../../data_hdd4t/pcap_process/pcap_spark/2020_01_10/中華電信/parquet/2020_01_10_country_session_withtime_v2.parquet/'
df = pd.read_parquet(in_parquet)
df #所有session
# kf = ks.read_parquet(in_parquet)
# df = kf

OSError: Passed non-file path: ../../data_hdd4t/pcap_process/pcap_spark/2020_01_10/中華電信/parquet/2020_01_10_country_session_withtime_v2.parquet/

In [351]:
df.head(10).to_excel('/home/antslab/Leo_code/Statistics/head.xlsx')

In [3]:
df = df.dropna()
df = df.reset_index(drop=True)
df.columns #所有欄位

Index(['frame.time_epoch', 'session_lifetime', 'session_packets',
       'tcp.window_size_value.avg', 'tcp.window_size_value.max', 'tcp.srcport',
       'tcp.dstport', 'tcp.flags.ack', 'tcp.flags.fin', 'tcp.flags.push',
       'tcp.flags.reset', 'tcp.flags.syn', 'ip.src', 'ip.dst', 'ip.ttl.avg',
       'ip.proto', 'frame.protocols.common', 'frame.len.sum', 'city',
       'subdivisions', 'latitude', 'longitude', 'domain', 'isp', 'network',
       'country'],
      dtype='object')

# 各欄位簡易統計分析

### Session類

In [267]:
df['session_lifetime'].value_counts() #seconds

0.000000      42700
0.000211       1424
0.000175       1411
0.000180       1406
0.000196       1370
              ...  
0.319105          1
1.276422          1
81.691391         1
0.319107          1
127.999466        1
Name: session_lifetime, Length: 436911, dtype: int64

In [268]:
df['session_lifetime'].value_counts().head(60) #seconds

0.000000    42700
0.000211     1424
0.000175     1411
0.000180     1406
0.000196     1370
0.000206     1363
0.000283     1359
0.000190     1340
0.000216     1335
0.000278     1323
0.000185     1322
0.000273     1319
0.000221     1318
0.000288     1302
0.000201     1295
0.000195     1286
0.000226     1278
0.000200     1276
0.000222     1264
0.000191     1259
0.000268     1252
0.000186     1240
0.000242     1227
0.000232     1221
0.000231     1207
0.000227     1202
0.000237     1199
0.000247     1195
0.000205     1191
0.000170     1187
0.000210     1178
0.000293     1176
0.000207     1175
0.000181     1174
0.000217     1156
0.000262     1145
0.000252     1140
0.000263     1135
0.000212     1128
0.000257     1128
0.000289     1117
0.000169     1116
0.000284     1112
0.000165     1107
0.000267     1107
0.000241     1103
0.000184     1102
0.000176     1102
0.000179     1101
0.000253     1097
0.000272     1093
0.000202     1087
0.000236     1080
0.000189     1076
0.000274     1075
0.000174  

In [269]:
df['session_packets'].value_counts() #各類別數量

2        281847
6        128060
4         46827
1         42700
3         32240
          ...  
1291          1
4878          1
781           1
4879          1
38931         1
Name: session_packets, Length: 3399, dtype: int64

In [270]:
df['session_packets'].value_counts().head(60)

2     281847
6     128060
4      46827
1      42700
3      32240
12     23990
10     21740
15     15400
44     12200
8      11971
14     10568
16     10162
7       7635
9       6998
18      6966
5       5944
21      5277
23      4216
20      4057
13      3972
11      3787
24      3777
31      3462
19      3324
30      3285
27      3274
36      3198
22      2793
33      2779
28      2611
17      2454
45      2035
32      1875
26      1872
25      1772
29      1714
42      1545
46      1422
39      1369
48      1338
34      1327
63      1201
76      1168
35      1167
51      1113
54      1110
37       995
43       984
40       914
38       885
69       829
57       827
75       821
60       794
88       788
47       747
66       707
50       699
72       681
41       662
Name: session_packets, dtype: int64

In [303]:
pd.DataFrame(df['session_packets'].value_counts().head(20))

,session_packets
2,281847
6,128060
4,46827
1,42700
3,32240
12,23990
10,21740
15,15400
44,12200
8,11971


In [288]:
df['ip.ttl.avg'].value_counts()

93.000000     70413
156.000000    43066
94.000000     35762
149.500000    27649
149.000000    27481
              ...  
91.308424         1
56.728983         1
92.022921         1
173.808383        1
100.953039        1
Name: ip.ttl.avg, Length: 41204, dtype: int64

In [289]:
df['ip.ttl.avg'].value_counts().head(60)

93.000000     70413
156.000000    43066
94.000000     35762
149.500000    27649
149.000000    27481
153.500000    23541
56.000000     18963
60.500000     18240
56.500000     15375
57.000000     14129
55.500000     12452
92.500000     11814
55.000000     11118
57.500000     10846
61.000000     10493
185.333333     9695
58.000000      9429
58.500000      8730
151.500000     8396
152.500000     7067
154.500000     6801
152.000000     6763
122.000000     6434
124.000000     6301
154.000000     5459
98.800000      5151
89.500000      4724
100.000000     4649
153.000000     4526
248.000000     4510
47.000000      4091
151.000000     4046
54.000000      3939
150.500000     3512
54.500000      3476
93.500000      3412
181.333333     3289
88.000000      3203
52.000000      3143
75.833333      3121
184.666667     3064
89.000000      2882
88.500000      2767
121.000000     2740
53.000000      2733
73.642857      2723
87.500000      2511
49.000000      2461
180.666667     2304
59.000000      2256


In [292]:
df['ip.proto'].value_counts(dropna=False)

6    797062
1      1538
Name: ip.proto, dtype: int64

In [274]:
df['tcp.flags.ack'].value_counts()

False    711802
True      86798
Name: tcp.flags.ack, dtype: int64

In [275]:
df['tcp.flags.fin'].value_counts()

False    770473
True      28127
Name: tcp.flags.fin, dtype: int64

In [276]:
df['tcp.flags.push'].value_counts()

False    797864
True        736
Name: tcp.flags.push, dtype: int64

In [277]:
df['tcp.flags.reset'].value_counts()

False    796532
True       2068
Name: tcp.flags.reset, dtype: int64

In [278]:
df['tcp.flags.syn'].value_counts()

True     711709
False     86891
Name: tcp.flags.syn, dtype: int64

### 攻擊者方(外部類/source)

In [266]:
df['tcp.window_size_value.max'].value_counts()

65535.0    137975
29200.0    123726
8192.0     117275
14600.0     77315
61690.0     55742
            ...  
20667.0         1
20669.0         1
1292.0          1
48970.0         1
32767.0         1
Name: tcp.window_size_value.max, Length: 29108, dtype: int64

In [264]:
df['tcp.window_size_value.max'].value_counts().head(60)

65535.0    137975
29200.0    123726
8192.0     117275
14600.0     77315
61690.0     55742
1200.0      50562
1024.0      31820
64240.0     17263
28960.0     14463
65340.0      4672
5840.0       2830
0.0          1694
64860.0      1471
64800.0      1205
64952.0      1176
64400.0       751
30016.0       658
229.0         629
16384.0       587
64380.0       451
65280.0       432
62920.0       424
24266.0       397
65392.0       385
1460.0        372
5440.0        357
29700.0       317
21402.0       273
32850.0       260
50555.0       236
63443.0       233
4202.0        219
26883.0       211
6001.0        205
34778.0       203
36368.0       200
3897.0        188
17520.0       178
36954.0       162
3616.0        158
64308.0       156
47916.0       149
20337.0       144
10467.0       142
8428.0        140
38402.0       140
24419.0       139
39938.0       137
65320.0       135
5808.0        131
61502.0       130
31088.0       130
51201.0       126
26460.0       123
56815.0       123
23094.0   

In [279]:
df['tcp.window_size_value.avg'].value_counts() #同session下windows size的平均值都一樣也很特別

4096.000000     92596
7300.000000     70860
30845.000000    55533
741.333333      46976
32767.500000    46464
                ...  
20149.333333        1
20020.500000        1
20021.000000        1
20021.500000        1
17042.500000        1
Name: tcp.window_size_value.avg, Length: 104545, dtype: int64

In [280]:
df['tcp.window_size_value.avg'].value_counts().head(60) #很巧的，大家的同session來回之平均windows size大小都一樣

4096.000000     92596
7300.000000     70860
30845.000000    55533
741.333333      46976
32767.500000    46464
512.000000      27031
47639.500000    25424
8192.000000     22536
13653.166667    16929
14600.000000     9042
4078.600000      7906
1024.000000      3950
8994.272727      3460
10474.666667     2769
2920.000000      2761
7602.600000      2624
31578.333333     2491
9189.800000      2343
9099.750000      1951
8874.583333      1857
9132.250000      1698
0.000000         1694
65535.000000     1539
32753.600000     1460
12446.428571     1397
7148.250000      1380
3059.666667      1339
9046.431818      1152
9058.409091      1136
18431.750000     1122
11782.333333     1100
12259.000000     1091
15121.428571     1071
44610.700000     1022
7088.391304       989
21544.000000      986
2760.750000       944
12608.928571      940
40833.857143      918
10074.666667      894
43912.250000      806
11734.666667      804
11786.000000      792
32054.000000      764
7856.000000       757
2977.77777

In [272]:
df['tcp.srcport'].value_counts() # 種類

445.0      28545
2842.0      8831
40103.0     5246
40112.0     5244
40144.0     3493
           ...  
43849.0        1
43857.0        1
25030.0        1
25029.0        1
45300.0        1
Name: tcp.srcport, Length: 64161, dtype: int64

In [273]:
df['tcp.srcport'].value_counts().head(60) #分布

445.0      28545
2842.0      8831
40103.0     5246
40112.0     5244
40144.0     3493
40416.0     3255
41938.0     3123
41928.0     3070
40240.0     2350
42017.0     1980
41957.0     1849
42108.0     1265
10978.0     1234
46868.0     1158
23.0        1018
1400.0       914
80.0         892
1024.0       886
139.0        785
7011.0       773
995.0        773
4434.0       765
587.0        763
2082.0       760
2087.0       759
2004.0       759
7002.0       758
465.0        755
3001.0       755
3389.0       753
2086.0       752
25.0         749
1443.0       741
3689.0       732
4567.0       731
52445.0      730
666.0        720
40173.0      713
34247.0      627
110.0        595
45286.0      540
55702.0      507
56889.0      435
41903.0      397
4935.0       339
56613.0      331
56773.0      293
9220.0       292
56711.0      287
9200.0       272
22.0         252
57239.0      244
12061.0      237
54140.0      233
55018.0      229
53362.0      227
34292.0      221
48546.0      215
15091.0      2

In [283]:
df['ip.src'].value_counts()

210.242.156.9      58025
210.242.237.131    49762
210.242.49.2       49634
5.154.238.169      46912
210.242.252.80     28408
                   ...  
49.206.31.40           1
183.82.115.38          1
14.248.102.181         1
110.54.246.64          1
47.37.90.133           1
Name: ip.src, Length: 68028, dtype: int64

In [284]:
df['ip.src'].value_counts().head(60)

210.242.156.9      58025
210.242.237.131    49762
210.242.49.2       49634
5.154.238.169      46912
210.242.252.80     28408
210.242.244.184    27828
210.242.237.127    10967
210.242.156.227     9536
210.242.154.92      9087
67.218.118.62       8828
91.217.254.167      8353
91.217.254.37       8305
210.242.156.142     5999
210.242.156.140     5848
220.188.55.202      5717
62.173.149.167      5335
62.173.149.87       5210
92.53.119.137       3619
110.169.74.183      3379
122.164.6.192       2746
111.162.139.122     2432
210.242.154.93      2209
211.75.51.34        1980
121.234.206.8       1731
80.82.70.187        1715
122.163.126.57      1300
211.22.246.103      1151
187.162.76.81       1063
217.59.107.44       1049
85.217.205.129       951
5.13.232.102         949
5.14.233.157         940
79.118.180.117       935
79.117.112.235       919
183.82.98.67         857
77.106.99.3          773
212.174.254.6        717
207.180.206.98       704
41.33.13.26          679
103.28.159.54        614


In [271]:
df.columns

Index(['frame.time_epoch', 'session_lifetime', 'session_packets',
       'tcp.window_size_value.avg', 'tcp.window_size_value.max', 'tcp.srcport',
       'tcp.dstport', 'tcp.flags.ack', 'tcp.flags.fin', 'tcp.flags.push',
       'tcp.flags.reset', 'tcp.flags.syn', 'ip.src', 'ip.dst', 'ip.ttl.avg',
       'ip.proto', 'frame.protocols.common', 'frame.len.sum', 'city',
       'subdivisions', 'latitude', 'longitude', 'domain', 'isp', 'network',
       'country'],
      dtype='object')

### Honeypot陷阱方(內部類/destination)

In [281]:
df['tcp.dstport'].value_counts()

23.0       175743
1433.0     165623
445.0      139774
80.0        21769
22.0        20105
            ...  
54368.0         1
61493.0         1
1456.0          1
47544.0         1
17405.0         1
Name: tcp.dstport, Length: 41978, dtype: int64

In [282]:
df['tcp.dstport'].value_counts().head(60)

23.0       175743
1433.0     165623
445.0      139774
80.0        21769
22.0        20105
5555.0      19922
8080.0      17411
4567.0      10202
81.0        10005
139.0        7393
3389.0       5518
2323.0       4101
8000.0       3212
443.0        2919
88.0         2758
9000.0       2636
60001.0      2563
26.0         1953
25.0         1756
21.0         1443
6379.0       1316
50070.0      1209
27017.0      1169
995.0        1161
13389.0      1152
50075.0      1112
37215.0      1087
110.0        1051
3128.0        968
587.0         952
666.0         947
2004.0        903
7002.0        894
11211.0       885
873.0         864
9001.0        852
465.0         831
85.0          821
1024.0        819
2082.0        812
5900.0        809
2086.0        808
4786.0        804
2087.0        801
1400.0        776
7011.0        767
4434.0        759
65529.0       757
3001.0        755
1443.0        741
3689.0        741
3306.0        707
5000.0        666
82.0          646
65533.0       615
83.0      

In [285]:
df['ip.dst'].value_counts()

210.242.146.122    56191
210.242.148.82      4427
210.242.145.85      4022
210.242.145.110     3866
210.242.145.80      3437
                   ...  
218.164.218.84         1
37.212.116.66          1
187.217.96.35          1
126.228.65.229         1
1.169.71.144           1
Name: ip.dst, Length: 3514, dtype: int64

In [286]:
df['ip.dst'].value_counts().head(60)

210.242.146.122    56191
210.242.148.82      4427
210.242.145.85      4022
210.242.145.110     3866
210.242.145.80      3437
210.242.148.64      3110
210.242.149.83      2558
210.242.146.108     2067
210.242.147.109     2062
210.242.147.99      1943
210.242.147.78      1941
210.242.145.119     1914
210.242.147.95      1904
210.242.148.102     1848
210.242.148.112     1816
210.242.149.92      1751
210.242.148.72      1729
210.242.145.64      1667
210.242.148.95      1632
210.242.147.106     1599
210.242.145.108     1539
210.242.149.120     1505
210.242.148.125     1495
210.242.148.83      1452
210.242.149.106     1452
210.242.148.109     1380
210.242.146.118     1341
210.242.149.103     1333
210.242.146.114     1249
210.242.149.70      1215
210.242.149.73      1211
210.242.146.103     1161
210.242.148.84      1126
210.242.146.111     1123
210.242.147.103     1119
210.242.149.79      1088
210.242.149.98      1059
210.242.146.64       971
210.242.146.117      943
210.242.149.66       923


In [237]:
df['network'].value_counts().head(60)

210.242.0.0/16      257927
5.154.238.0/24       46912
91.217.254.0/24      16658
71.6.128.0/17        12420
62.173.144.0/20      10554
196.52.43.0/24        9293
67.218.118.0/23       8828
216.218.204.0/22      7701
36.224.0.0/12         7412
114.32.0.0/12         7256
1.160.0.0/12          5886
220.188.0.0/16        5738
139.162.96.0/19       5346
92.53.112.0/21        3619
110.168.0.0/15        3467
111.240.0.0/12        3211
197.32.0.0/11         3156
222.252.0.0/15        3053
172.104.64.0/18       2974
122.164.0.0/18        2750
31.163.128.0/18       2555
184.105.246.0/23      2497
5.63.144.0/21         2460
88.202.184.0/21       2449
111.162.0.0/15        2447
220.132.0.0/14        2438
80.82.70.0/24         2431
156.192.0.0/11        2391
223.204.0.0/14        2313
218.160.0.0/12        2257
104.206.128.0/22      2207
211.75.0.0/16         2198
125.224.0.0/13        2103
222.254.0.0/16        2071
220.136.0.0/13        2029
121.232.0.0/14        1754
118.160.0.0/13        1687
4

In [301]:
df[df['frame.time_epoch']==1578586232.931816]#.columns

,frame.time_epoch,session_lifetime,session_packets,tcp.window_size_value.avg,tcp.window_size_value.max,tcp.srcport,tcp.dstport,tcp.flags.ack,tcp.flags.fin,tcp.flags.push,...,frame.protocols.common,frame.len.sum,city,subdivisions,latitude,longitude,domain,isp,network,country
756759,1.578586e+09,0.0,1,29200.0,29200.0,23.0,17622.0,True,False,False,...,eth:ethertype:ip:icmp:ip:tcp,86,Auburn,Indiana,41.359,-85.0468,sbcglobal.net,AT&T U-verse,162.197.192.0/19,United States


In [56]:
#ip check switch
file = open('./hp_ips.txt','r')
ip_li = file.readlines()
ips_li = []
for ip in ip_li:
    ips_li.append(".".join(ip.split()[-1].split('.')[:3]))
ips_li    

['61.66.51',
 '220.229.172',
 '210.64.37',
 '211.73.131',
 '210.241.214',
 '203.105.227',
 '203.105.228',
 '203.105.229',
 '203.105.230',
 '203.105.231',
 '210.200.17',
 '210.202.233',
 '210.203.16',
 '210.63.253',
 '222.156.247',
 '61.64.230',
 '61.62.66',
 '61.62.173',
 '219.84.227',
 '219.85.81',
 '103.224.192',
 '103.224.193',
 '103.225.3',
 '175.96.14',
 '175.96.15',
 '175.96.48',
 '175.96.49',
 '175.96.13',
 '61.67.24',
 '43.255.88',
 '43.255.90',
 '43.255.181',
 '43.255.183',
 '202.144.213',
 '210.242.145',
 '210.242.146',
 '210.242.147',
 '210.242.148',
 '210.242.149',
 '203.187.87']

In [57]:
%%time
df1 = df[['tcp.dstport','domain','ip.src','frame.time_epoch']] # 需要加入time
df1['tcp.dstport'] = df1['tcp.dstport'].astype(int)
df1 = df1.sort_values('frame.time_epoch')
# df1 = df1[1000:]
df1 = df1.reset_index(drop=True)
temp = df1['frame.time_epoch'].tolist()
del temp[0]
temp.append(0)
df1['frame.time_epoch2'] = pd.Series(temp)
df1['frame.time_epoch2'] = df1['frame.time_epoch2'] - df1['frame.time_epoch']
temp = df1['frame.time_epoch2'].tolist()
del temp[-1]
time_avg = np.mean(temp)
print('###Time interval average:',time_avg,'###')
df1

/home/antslab/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


###Time interval average: 0.10815727067760171 ###
CPU times: user 238 ms, sys: 20.6 ms, total: 259 ms
Wall time: 252 ms


,tcp.dstport,domain,ip.src,frame.time_epoch,frame.time_epoch2
0,62035,hinet.net,210.242.147.65,1.578585e+09,2.098083e-05
1,62036,hinet.net,210.242.149.113,1.578585e+09,5.888939e-05
2,62030,hinet.net,210.242.148.117,1.578585e+09,2.002716e-05
3,62031,hinet.net,210.242.149.109,1.578585e+09,1.615047e-03
4,1433,hinet.net,210.242.154.92,1.578585e+09,2.739429e-04
...,...,...,...,...,...
798595,445,hinet.net,210.242.156.9,1.578671e+09,6.618500e-04
798596,1433,bisv.ru,158.58.130.151,1.578671e+09,1.395061e-01
798597,1433,hinet.net,210.242.156.9,1.578671e+09,1.885486e-02
798598,1433,hinet.net,210.242.156.9,1.578671e+09,1.121531e-01


In [140]:
time_avg = np.median(temp)
time_avg

0.05910301208496094

In [150]:
iqr = np.quantile(time_avg,0.75) - np.quantile(time_avg,0.25)
time_avg = np.quantile(time_avg,0.75) + 3*iqr
time_avg

0.05910301208496094

In [152]:
# counts = np.bincount(time_avg)
# np.argmax(counts)
from scipy import stats

print("眾數:",stats.mode(time_avg)[0][0])
time_avg = np.mean(temp)

眾數: 0.05910301208496094


In [61]:
df1gb = df1.groupby(['tcp.dstport','domain'])
all_groups = df1gb.groups

In [65]:
all_groups_dict = {}
for i,(k,v) in enumerate(all_groups.items()):
    all_groups_dict[k] = i+1
len(all_groups_dict)    

80583

In [102]:
def func(gb):
    return all_groups_dict[tuple(gb)]
def func2(gb):
    return list(gb)
# res = df1gb.apply(func)
sub1 = df1gb.aggregate(lambda tdf: tdf.tolist())
# res = df1gb['frame.time_epoch2'].transform(func2)
sub1

ip.src  \
tcp.dstport domain                                                                 
0           ridgewireless.net  [67.218.118.62, 67.218.118.62, 67.218.118.62, ...   
            vhost.live                                           [5.154.238.169]   
1           hostpark.com.ua    [91.217.254.167, 91.217.254.167, 91.217.254.16...   
            ip-151-80-47.eu                                        [151.80.47.9]   
            ridgewireless.net  [67.218.118.62, 67.218.118.62, 67.218.118.62, ...   
...                                                                          ...   
65533       hinet.net                                           [210.242.149.67]   
            megared.net.mx                                      [210.242.146.89]   
65534       itnet33.ru                                         [210.242.149.116]   
            totinternet.net                                    [210.242.147.126]   
65535       vhost.live                            [5.154.238.169, 5.154.238.169]   

                                                                frame.time_epoch  \
tcp.dstport domain                                                                 
0           ridgewireless.net  [1578586723.338771, 1578587040.200059, 1578587...   
            vhost.live                                       [1578586550.129609]   
1           hostpark.com.ua    [1578585737.539697, 1578592052.698103, 1578594...   
            ip-151-80-47.eu                                  [1578664237.399538]   
            ridgewireless.net  [1578586684.543369, 1578586917.312086, 1578587...   
...                                                                          ...   
65533       hinet.net                                         [1578601327.88702]   
            megared.net.mx                                   [1578592372.574641]   
65534       itnet33.ru                                       [1578596742.760882]   
            totinternet.net                                  [1578607262.937732]   
65535       vhost.live                     [1578586419.530024, 1578587901.48934]   

                                                               frame.time_epoch2  
tcp.dstport domain                                                                
0           ridgewireless.net  [0.002071857452392578, 0.014052152633666992, 0...  
            vhost.live                                    [0.036720991134643555]  
1           hostpark.com.ua    [0.11132407188415527, 0.19301104545593262, 0.0...  
            ip-151-80-47.eu                                [0.14766192436218262]  
            ridgewireless.net  [0.021357059478759766, 0.0001418590545654297, ...  
...                                                                          ...  
65533       hinet.net                                      [0.04028487205505371]  
            megared.net.mx                               [0.0002930164337158203]  
65534       itnet33.ru                                     [0.17230701446533203]  
            totinternet.net                                [0.29064512252807617]  
65535       vhost.live                [0.03666496276855469, 0.04424786567687988]  

[80583 rows x 3 columns]

In [117]:
def only_one(s):
    s = np.unique(s)
    if len(s)<230:
        return np.nan
    else:
        return len(s)
sub1['count'] = sub1['ip.src'].map(only_one)
sub11 = sub1.dropna()
sub11

ip.src  \
tcp.dstport domain                                                                     
23          axtel.net              [189.209.251.88, 187.162.140.59, 200.76.195.18...   
            comcast.net            [73.155.104.254, 24.18.193.186, 73.223.250.246...   
            gvt.net.br             [187.113.186.176, 189.27.8.16, 179.162.213.64,...   
            hinet.net              [36.234.135.144, 210.242.252.80, 36.226.170.53...   
            prod-infinitum.com.mx  [189.228.76.59, 189.135.177.118, 189.188.184.2...   
            rapid7.com             [71.6.233.63, 71.6.233.156, 71.6.233.209, 71.6...   
            speedy.com.ar          [181.21.151.104, 190.175.47.115, 190.48.79.182...   
            tedata.net             [156.222.211.211, 81.10.25.104, 41.235.198.174...   
            telecomitalia.it       [79.24.197.108, 82.186.120.234, 80.181.143.107...   
            telesp.net.br          [179.111.147.94, 177.138.47.83, 189.69.54.209,...   
            ttnet.com.tr           [85.97.187.12, 88.249.56.14, 85.105.241.172, 7...   
            virtua.com.br          [191.182.229.209, 187.65.223.56, 179.217.230.6...   
            vivozap.com.br         [179.113.99.15, 191.23.19.182, 191.205.79.112,...   
25          rapid7.com             [71.6.233.26, 71.6.233.106, 71.6.233.17, 71.6....   
80          prod-infinitum.com.mx  [201.103.79.111, 201.110.133.123, 189.142.228....   
            telecomitalia.it       [62.86.66.174, 80.20.240.215, 82.62.141.254, 8...   
            telesp.net.br          [189.69.18.251, 187.75.214.114, 179.98.39.224,...   
            ttnet.com.tr           [88.224.49.46, 78.187.104.250, 78.169.76.152, ...   
            vivozap.com.br         [189.0.47.20, 191.205.39.85, 191.23.11.41, 191...   
81          hinet.net              [114.34.222.49, 114.33.188.183, 114.32.40.185,...   
            prod-infinitum.com.mx  [187.173.201.14, 201.103.31.169, 201.103.232.4...   
            ttnet.com.tr           [78.186.49.146, 85.105.35.176, 88.248.160.189,...   
110         rapid7.com             [71.6.233.63, 71.6.233.180, 71.6.233.17, 71.6....   
139         rapid7.com             [71.6.233.174, 71.6.233.196, 71.6.233.46, 71.6...   
445         PMPL-Broadband.net     [43.251.172.58, 43.251.172.112, 43.251.172.98,...   
            actcorp.in             [183.83.133.75, 49.207.136.198, 183.82.111.6, ...   
            cantv.net              [201.248.152.15, 201.211.172.96, 200.109.66.20...   
            hinet.net              [210.242.237.131, 210.242.156.227, 210.242.49....   
            pldt.net               [122.52.48.243, 58.69.17.4, 122.52.48.243, 49....   
            rapid7.com             [71.6.233.130, 71.6.233.119, 71.6.233.179, 71....   
            telkom.net.id          [110.138.151.213, 125.164.9.98, 110.137.13.184...   
            totinternet.net        [182.52.134.8, 1.1.132.18, 125.26.165.10, 125....   
            viettel.vn             [115.79.235.252, 171.241.85.7, 27.72.243.103, ...   
            vnpt.vn                [14.190.205.125, 113.173.202.252, 14.181.127.2...   
465         rapid7.com             [71.6.233.174, 71.6.233.75, 71.6.233.215, 71.6...   
587         rapid7.com             [71.6.233.33, 71.6.233.15, 71.6.233.32, 71.6.2...   
666         rapid7.com             [71.6.233.218, 71.6.233.43, 71.6.233.18, 71.6....   
995         rapid7.com             [71.6.233.63, 71.6.233.15, 71.6.233.8, 71.6.23...   
1024        rapid7.com             [71.6.233.247, 71.6.233.251, 71.6.233.69, 71.6...   
1400        rapid7.com             [71.6.233.113, 71.6.233.78, 71.6.233.121, 71.6...   
1443        rapid7.com             [71.6.233.80, 71.6.233.164, 71.6.233.153, 71.6...   
2004        rapid7.com             [71.6.233.158, 71.6.233.120, 71.6.233.68, 71.6...   
2082        rapid7.com             [71.6.233.166, 71.6.233.214, 71.6.233.236, 71....   
2086        rapid7.com             [71.6.233.145, 71.6.233.131, 71.6.233.3, 71.6....   
2087        rapid7.com        

In [122]:
sub1_idx_li = sub11.index.tolist() #tuple
sub1_ips_li = sub11['ip.src'].tolist() #list of list
sub1_time_li = sub11['frame.time_epoch'].tolist() #list of list
sub1_elapse_li = sub11['frame.time_epoch2'].tolist() #list of list

In [133]:
new_idx = []
new_ips = []
new_time = []
new_elapse = []
new_events = []
for i,(idx,ips_li,time_li,elapse_li) in tqdm(enumerate(zip(sub1_idx_li,sub1_ips_li,sub1_time_li,sub1_elapse_li))):
    k=1
    event = 's1_c'+str(i+1)+'_e'#+str(k)
    for ip,time,elapse in zip(ips_li,time_li,elapse_li):
        event_ = event + str(k)
        new_idx.append(idx)
        new_ips.append(ip)
        new_time.append(time)
        new_events.append(event_)
        if elapse>time_avg:
            k = k+1
    

58it [00:00, 454.51it/s]


In [134]:
sub12 = pd.DataFrame(list(zip(new_idx, new_ips,new_time,new_events)),
                          columns=['ClusterName','src_ip','packet_time','event_id'])
sub12

,ClusterName,src_ip,packet_time,event_id
0,"(23, axtel.net)",189.209.251.88,1.578585e+09,s1_c1_e1
1,"(23, axtel.net)",187.162.140.59,1.578585e+09,s1_c1_e1
2,"(23, axtel.net)",200.76.195.189,1.578585e+09,s1_c1_e1
3,"(23, axtel.net)",189.209.174.124,1.578585e+09,s1_c1_e1
4,"(23, axtel.net)",187.167.64.56,1.578585e+09,s1_c1_e1
...,...,...,...,...
227762,"(8080, vivozap.com.br)",152.250.142.238,1.578670e+09,s1_c58_e114
227763,"(8080, vivozap.com.br)",191.17.62.91,1.578671e+09,s1_c58_e114
227764,"(8080, vivozap.com.br)",191.23.4.77,1.578671e+09,s1_c58_e115
227765,"(8080, vivozap.com.br)",191.205.89.203,1.578671e+09,s1_c58_e116


In [323]:
sub12[sub12['ClusterName']==(80,'vivozap.com.br')]

,ClusterName,src_ip,packet_time,event_id
117479,"(80, vivozap.com.br)",189.0.47.20,1.578585e+09,s1_c19_e1
117480,"(80, vivozap.com.br)",191.205.39.85,1.578585e+09,s1_c19_e1
117481,"(80, vivozap.com.br)",191.23.11.41,1.578585e+09,s1_c19_e2
117482,"(80, vivozap.com.br)",191.23.93.198,1.578586e+09,s1_c19_e3
117483,"(80, vivozap.com.br)",191.8.58.184,1.578586e+09,s1_c19_e4
...,...,...,...,...
117726,"(80, vivozap.com.br)",152.250.136.183,1.578670e+09,s1_c19_e106
117727,"(80, vivozap.com.br)",152.250.78.71,1.578671e+09,s1_c19_e106
117728,"(80, vivozap.com.br)",191.23.118.101,1.578671e+09,s1_c19_e107
117729,"(80, vivozap.com.br)",152.249.54.226,1.578671e+09,s1_c19_e108


In [324]:
sub12[sub12['event_id']=='s1_c19_e106']

,ClusterName,src_ip,packet_time,event_id
117720,"(80, vivozap.com.br)",191.17.196.95,1.578668e+09,s1_c19_e106
117721,"(80, vivozap.com.br)",191.8.41.237,1.578669e+09,s1_c19_e106
117722,"(80, vivozap.com.br)",179.228.47.133,1.578669e+09,s1_c19_e106
117723,"(80, vivozap.com.br)",189.97.72.19,1.578669e+09,s1_c19_e106
117724,"(80, vivozap.com.br)",191.19.132.55,1.578670e+09,s1_c19_e106
117725,"(80, vivozap.com.br)",191.8.74.18,1.578670e+09,s1_c19_e106
117726,"(80, vivozap.com.br)",152.250.136.183,1.578670e+09,s1_c19_e106
117727,"(80, vivozap.com.br)",152.250.78.71,1.578671e+09,s1_c19_e106


In [329]:
sub12.iloc[117723,2]

1578669420.575357

In [330]:
df[df['frame.time_epoch']==1578669420.575357]

,frame.time_epoch,session_lifetime,session_packets,tcp.window_size_value.avg,tcp.window_size_value.max,tcp.srcport,tcp.dstport,tcp.flags.ack,tcp.flags.fin,tcp.flags.push,...,frame.protocols.common,frame.len.sum,city,subdivisions,latitude,longitude,domain,isp,network,country
27887,1.578669e+09,1.243467,13,7009.538462,29200.0,53668.0,80.0,False,False,False,...,eth:ethertype:ip:tcp,1132,São Paulo,Sao Paulo,-23.63,-46.6322,vivozap.com.br,Vivo,189.97.72.0/21,Brazil


In [169]:
def toosmall(s):
    if s<26:
        return np.nan
    else:
        return s
sub121 = sub12.groupby('event_id').count()
sub121['tooSmall'] = sub121['src_ip'].map(toosmall)
sub121 = sub121.dropna()
case_eventid = sub121.index.tolist()
sub121

,ClusterName,src_ip,packet_time,tooSmall
event_id,,,,
s1_c11_e21,52,52,52,52.0
s1_c1_e16,28,28,28,28.0
s1_c20_e7,26,26,26,26.0
s1_c23_e149,80,80,80,80.0
s1_c27_e10,34,34,34,34.0
s1_c28_e10364,132,132,132,132.0
s1_c28_e10366,387,387,387,387.0
s1_c28_e10368,375,375,375,375.0
s1_c28_e10956,36,36,36,36.0


In [170]:
case_eventid

['s1_c11_e21',
 's1_c1_e16',
 's1_c20_e7',
 's1_c23_e149',
 's1_c27_e10',
 's1_c28_e10364',
 's1_c28_e10366',
 's1_c28_e10368',
 's1_c28_e10956',
 's1_c28_e11020',
 's1_c28_e11611',
 's1_c28_e11613',
 's1_c28_e11614',
 's1_c28_e11792',
 's1_c28_e12022',
 's1_c28_e13139',
 's1_c28_e14',
 's1_c28_e14198',
 's1_c28_e14200',
 's1_c28_e15996',
 's1_c28_e19345',
 's1_c28_e231',
 's1_c28_e2392',
 's1_c28_e255',
 's1_c28_e298',
 's1_c28_e299',
 's1_c28_e301',
 's1_c28_e302',
 's1_c28_e3667',
 's1_c28_e3673',
 's1_c28_e3725',
 's1_c28_e391',
 's1_c28_e4',
 's1_c28_e7875',
 's1_c28_e9',
 's1_c28_e9093',
 's1_c36_e132',
 's1_c3_e12',
 's1_c4_e18528',
 's1_c4_e18532',
 's1_c4_e35107',
 's1_c4_e493',
 's1_c4_e5107',
 's1_c4_e5174',
 's1_c4_e5175',
 's1_c4_e618',
 's1_c52_e32',
 's1_c5_e9',
 's1_c8_e51',
 's1_c8_e66']

In [172]:
sub122 = sub12[sub12.event_id.isin(case_eventid)]
sub122 = sub122.reset_index(drop=True)
sub122

,ClusterName,src_ip,packet_time,event_id
0,"(23, axtel.net)",187.162.6.11,1.578586e+09,s1_c1_e16
1,"(23, axtel.net)",187.178.71.154,1.578586e+09,s1_c1_e16
2,"(23, axtel.net)",187.177.32.229,1.578586e+09,s1_c1_e16
3,"(23, axtel.net)",189.205.154.186,1.578587e+09,s1_c1_e16
4,"(23, axtel.net)",189.208.62.91,1.578587e+09,s1_c1_e16
...,...,...,...,...
8008,"(5555, netvigator.com)",112.120.225.34,1.578587e+09,s1_c52_e32
8009,"(5555, netvigator.com)",219.73.94.96,1.578587e+09,s1_c52_e32
8010,"(5555, netvigator.com)",58.153.25.190,1.578587e+09,s1_c52_e32
8011,"(5555, netvigator.com)",219.79.78.61,1.578587e+09,s1_c52_e32


In [259]:
sub122[sub122.ClusterName == (5555, 'netvigator.com')]

,ClusterName,src_ip,packet_time,event_id,max_time
7938,"(5555, netvigator.com)",42.3.104.54,1.578586e+09,s1_c52_e32,NaN
7939,"(5555, netvigator.com)",58.153.222.133,1.578586e+09,s1_c52_e32,NaN
7940,"(5555, netvigator.com)",218.250.24.207,1.578586e+09,s1_c52_e32,NaN
7941,"(5555, netvigator.com)",219.78.24.28,1.578586e+09,s1_c52_e32,NaN
7942,"(5555, netvigator.com)",219.73.100.167,1.578586e+09,s1_c52_e32,NaN
...,...,...,...,...,...
8008,"(5555, netvigator.com)",112.120.225.34,1.578587e+09,s1_c52_e32,NaN
8009,"(5555, netvigator.com)",219.73.94.96,1.578587e+09,s1_c52_e32,NaN
8010,"(5555, netvigator.com)",58.153.25.190,1.578587e+09,s1_c52_e32,NaN
8011,"(5555, netvigator.com)",219.79.78.61,1.578587e+09,s1_c52_e32,NaN


In [207]:
# sub122.packet_time
pd.to_datetime(sub122.packet_time,unit='s')

0      2020-01-09 16:10:35.583620071
1      2020-01-09 16:14:19.798810005
2      2020-01-09 16:14:38.694298983
3      2020-01-09 16:15:18.452208996
4      2020-01-09 16:16:23.165247917
                    ...             
8008   2020-01-09 16:25:17.677247047
8009   2020-01-09 16:26:03.996927023
8010   2020-01-09 16:26:15.961038113
8011   2020-01-09 16:26:21.105045080
8012   2020-01-09 16:26:21.483767033
Name: packet_time, Length: 8013, dtype: datetime64[ns]

In [178]:
# sub122['time_normed'] = df.groupby('A')['C'].transform( lambda x: (x-min(x))/(max(x)-min(x)))
max_time = sub122.groupby('event_id')['packet_time'].agg('max')
min_time = sub122.groupby('event_id')['packet_time'].agg('min')
sub123 = pd.concat([max_time,min_time],axis=1)
sub123.columns = ['max_time','min_time']
# type(max_time)
sub123


,max_time,min_time
event_id,,
s1_c11_e21,1.578587e+09,1.578586e+09
s1_c1_e16,1.578588e+09,1.578586e+09
s1_c20_e7,1.578587e+09,1.578586e+09
s1_c23_e149,1.578587e+09,1.578586e+09
s1_c27_e10,1.578587e+09,1.578586e+09
s1_c28_e10364,1.578627e+09,1.578627e+09
s1_c28_e10366,1.578627e+09,1.578627e+09
s1_c28_e10368,1.578627e+09,1.578627e+09
s1_c28_e10956,1.578630e+09,1.578630e+09


sub-space2 (activity2)

In [179]:
df.columns

Index(['frame.time_epoch', 'session_lifetime', 'session_packets',
       'tcp.window_size_value.avg', 'tcp.window_size_value.max', 'tcp.srcport',
       'tcp.dstport', 'tcp.flags.ack', 'tcp.flags.fin', 'tcp.flags.push',
       'tcp.flags.reset', 'tcp.flags.syn', 'ip.src', 'ip.dst', 'ip.ttl.avg',
       'ip.proto', 'frame.protocols.common', 'frame.len.sum', 'city',
       'subdivisions', 'latitude', 'longitude', 'domain', 'isp', 'network',
       'country'],
      dtype='object')

In [180]:
df2 = df[['tcp.srcport','frame.protocols.common','country','frame.time_epoch']] # 需要加入time
df2['tcp.srcport'] = df2['tcp.srcport'].astype(int)
df2 = df2.sort_values('frame.time_epoch')
# df1 = df1[1000:]
df2 = df2.reset_index(drop=True)
temp = df2['frame.time_epoch'].tolist()
del temp[0]
temp.append(0)
df2['frame.time_epoch2'] = pd.Series(temp)
df2['frame.time_epoch2'] = df2['frame.time_epoch2'] - df2['frame.time_epoch']
temp = df2['frame.time_epoch2'].tolist()
del temp[-1]
time_avg = np.mean(temp)
print('###Time interval average:',time_avg,'###')
df2

/home/antslab/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


###Time interval average: 0.10815727067760171 ###


,tcp.srcport,frame.protocols.common,country,frame.time_epoch,frame.time_epoch2
0,1433,eth:ethertype:ip:tcp,Taiwan,1.578585e+09,2.098083e-05
1,1433,eth:ethertype:ip:tcp,Taiwan,1.578585e+09,5.888939e-05
2,1433,eth:ethertype:ip:tcp,Taiwan,1.578585e+09,2.002716e-05
3,1433,eth:ethertype:ip:tcp,Taiwan,1.578585e+09,1.615047e-03
4,62032,eth:ethertype:ip:tcp,Taiwan,1.578585e+09,2.739429e-04
...,...,...,...,...,...
798595,51391,eth:ethertype:ip:tcp,Taiwan,1.578671e+09,6.618500e-04
798596,55651,eth:ethertype:ip:tcp,Russia,1.578671e+09,1.395061e-01
798597,51770,eth:ethertype:ip:tcp,Taiwan,1.578671e+09,1.885486e-02
798598,51944,eth:ethertype:ip:tcp,Taiwan,1.578671e+09,1.121531e-01


In [182]:
df2gb = df2.groupby(['tcp.srcport','frame.protocols.common'])
all_groups = df2gb.groups
all_groups_dict = {}
for i,(k,v) in enumerate(all_groups.items()):
    all_groups_dict[k] = i+1
print(len(all_groups_dict))
sub2 = df2gb.aggregate(lambda tdf: tdf.tolist())
sub2

64203


,,country,frame.time_epoch,frame.time_epoch2
tcp.srcport,frame.protocols.common,,,
0,eth:ethertype:ip:tcp,"[Venezuela, Bangladesh, Bangladesh, Venezuela,...","[1578585098.034262, 1578585208.464728, 1578585...","[0.05766916275024414, 0.01983189582824707, 0.0..."
1,eth:ethertype:ip:tcp,"[Romania, Taiwan, Australia]","[1578586731.452175, 1578613587.303698, 1578651...","[0.02097320556640625, 0.2985038757324219, 0.02..."
2,eth:ethertype:ip:tcp,"[United States, Paraguay, United States, Unite...","[1578590471.225098, 1578604586.660513, 1578610...","[0.03829312324523926, 0.023658037185668945, 0...."
3,eth:ethertype:ip:tcp,"[United States, Canada, United States, United ...","[1578590647.260465, 1578594076.433438, 1578629...","[0.29134702682495117, 0.0938880443572998, 0.04..."
4,eth:ethertype:ip:tcp,"[Romania, United States, Taiwan, Australia, Un...","[1578587072.285621, 1578625276.316393, 1578639...","[0.00854802131652832, 0.18378520011901855, 0.0..."
...,...,...,...,...
65531,eth:ethertype:ip:tcp,"[Taiwan, Taiwan, Brazil, Taiwan, Taiwan, Unite...","[1578593965.301949, 1578594304.178018, 1578599...","[0.0002129077911376953, 0.01293492317199707, 0..."
65532,eth:ethertype:ip:tcp,"[Taiwan, Taiwan, Taiwan, Taiwan, United States...","[1578591629.4491, 1578599358.443316, 157860197...","[0.0003058910369873047, 0.014976024627685547, ..."
65533,eth:ethertype:ip:tcp,"[Pakistan, Romania, Romania, Romania, Taiwan, ...","[1578585766.495634, 1578586494.184619, 1578586...","[0.06252503395080566, 0.036730051040649414, 0...."


In [214]:
df2['frame.protocols.common'].unique()

array(['eth:ethertype:ip:tcp', 'eth:ethertype:ip:icmp:ip:tcp',
       'eth:ethertype:ip:icmp:ip:tcp:http',
       'eth:ethertype:ip:icmp:ip:tcp:http:data',
       'eth:ethertype:ip:icmp:ip:tcp:ssh'], dtype=object)

In [333]:
kk = sub2.index.tolist()
for k in kk:
    if 'http' in k[1]:
        print(k)

(80, 'eth:ethertype:ip:icmp:ip:tcp:http')
(80, 'eth:ethertype:ip:icmp:ip:tcp:http:data')


In [334]:
def only_few(s):
    s = np.unique(s)
    if len(s)<2:
        return np.nan
    else:
        return len(s)
sub2['count'] = sub2['country'].map(only_few)
sub21 = sub2.dropna()
sub21

,,country,frame.time_epoch,frame.time_epoch2,count
tcp.srcport,frame.protocols.common,,,,
0,eth:ethertype:ip:tcp,"[Venezuela, Bangladesh, Bangladesh, Venezuela,...","[1578585098.034262, 1578585208.464728, 1578585...","[0.05766916275024414, 0.01983189582824707, 0.0...",7.0
1,eth:ethertype:ip:tcp,"[Romania, Taiwan, Australia]","[1578586731.452175, 1578613587.303698, 1578651...","[0.02097320556640625, 0.2985038757324219, 0.02...",3.0
2,eth:ethertype:ip:tcp,"[United States, Paraguay, United States, Unite...","[1578590471.225098, 1578604586.660513, 1578610...","[0.03829312324523926, 0.023658037185668945, 0....",4.0
3,eth:ethertype:ip:tcp,"[United States, Canada, United States, United ...","[1578590647.260465, 1578594076.433438, 1578629...","[0.29134702682495117, 0.0938880443572998, 0.04...",5.0
4,eth:ethertype:ip:tcp,"[Romania, United States, Taiwan, Australia, Un...","[1578587072.285621, 1578625276.316393, 1578639...","[0.00854802131652832, 0.18378520011901855, 0.0...",5.0
...,...,...,...,...,...
65531,eth:ethertype:ip:tcp,"[Taiwan, Taiwan, Brazil, Taiwan, Taiwan, Unite...","[1578593965.301949, 1578594304.178018, 1578599...","[0.0002129077911376953, 0.01293492317199707, 0...",5.0
65532,eth:ethertype:ip:tcp,"[Taiwan, Taiwan, Taiwan, Taiwan, United States...","[1578591629.4491, 1578599358.443316, 157860197...","[0.0003058910369873047, 0.014976024627685547, ...",3.0
65533,eth:ethertype:ip:tcp,"[Pakistan, Romania, Romania, Romania, Taiwan, ...","[1578585766.495634, 1578586494.184619, 1578586...","[0.06252503395080566, 0.036730051040649414, 0....",4.0


In [185]:
sub21

3.0

In [335]:
sub2_idx_li = sub21.index.tolist() #tuple2
sub2_country_li = sub21['country'].tolist() #list of list
sub2_time_li = sub21['frame.time_epoch'].tolist() #list of list
sub2_elapse_li = sub21['frame.time_epoch2'].tolist() #list of list

new_idx = []
new_cou = []
new_time = []
new_elapse = []
new_events = []
for i,(idx,cou_li,time_li,elapse_li) in tqdm(enumerate(zip(sub2_idx_li,sub2_country_li,sub2_time_li,sub2_elapse_li))):
    k=1
    event = 's2_c'+str(i+1)+'_e'#+str(k)
    for country,time,elapse in zip(cou_li,time_li,elapse_li):
        event_ = event + str(k)
        new_idx.append(idx)
        new_cou.append(country)
        new_time.append(time)
        new_events.append(event_)
        if elapse>time_avg:
            k = k+1

sub22 = pd.DataFrame(list(zip(new_idx, new_cou,new_time,new_events)),
                          columns=['ClusterName','country','packet_time','event_id'])
sub22            

58119it [00:00, 112416.44it/s]


,ClusterName,country,packet_time,event_id
0,"(0, eth:ethertype:ip:tcp)",Venezuela,1.578585e+09,s2_c1_e1
1,"(0, eth:ethertype:ip:tcp)",Bangladesh,1.578585e+09,s2_c1_e1
2,"(0, eth:ethertype:ip:tcp)",Bangladesh,1.578586e+09,s2_c1_e1
3,"(0, eth:ethertype:ip:tcp)",Venezuela,1.578587e+09,s2_c1_e1
4,"(0, eth:ethertype:ip:tcp)",Bangladesh,1.578588e+09,s2_c1_e1
...,...,...,...,...
784991,"(65535, eth:ethertype:ip:tcp)",United States,1.578659e+09,s2_c58119_e6
784992,"(65535, eth:ethertype:ip:tcp)",United States,1.578659e+09,s2_c58119_e6
784993,"(65535, eth:ethertype:ip:tcp)",United States,1.578659e+09,s2_c58119_e6
784994,"(65535, eth:ethertype:ip:tcp)",United States,1.578660e+09,s2_c58119_e7


In [336]:
sub22[sub22.ClusterName==(80, 'eth:ethertype:ip:icmp:ip:tcp:http:data')]

,ClusterName,country,packet_time,event_id
2987,"(80, eth:ethertype:ip:icmp:ip:tcp:http:data)",Turkey,1.578605e+09,s2_c83_e1
2988,"(80, eth:ethertype:ip:icmp:ip:tcp:http:data)",Brazil,1.578618e+09,s2_c83_e2
2989,"(80, eth:ethertype:ip:icmp:ip:tcp:http:data)",India,1.578618e+09,s2_c83_e2
2990,"(80, eth:ethertype:ip:icmp:ip:tcp:http:data)",United States,1.578634e+09,s2_c83_e2
2991,"(80, eth:ethertype:ip:icmp:ip:tcp:http:data)",Poland,1.578646e+09,s2_c83_e2
2992,"(80, eth:ethertype:ip:icmp:ip:tcp:http:data)",Indonesia,1.578668e+09,s2_c83_e2


In [346]:
sub22.iloc[2992,2]

1578667959.944518

In [347]:
df[df['frame.time_epoch']==1578667959.944518]

,frame.time_epoch,session_lifetime,session_packets,tcp.window_size_value.avg,tcp.window_size_value.max,tcp.srcport,tcp.dstport,tcp.flags.ack,tcp.flags.fin,tcp.flags.push,...,frame.protocols.common,frame.len.sum,city,subdivisions,latitude,longitude,domain,isp,network,country
141782,1.578668e+09,104.75882,111,235.0,235.0,80.0,56074.0,True,False,False,...,eth:ethertype:ip:icmp:ip:tcp:http:data,65490,Jakarta,Jakarta,-6.1741,106.8296,iconpln.net.id,PT Indonesia Comnets Plus,223.25.96.0/21,Indonesia


In [322]:
df[df['frame.time_epoch']==1578655831.011847]#.columns

,frame.time_epoch,session_lifetime,session_packets,tcp.window_size_value.avg,tcp.window_size_value.max,tcp.srcport,tcp.dstport,tcp.flags.ack,tcp.flags.fin,tcp.flags.push,...,frame.protocols.common,frame.len.sum,city,subdivisions,latitude,longitude,domain,isp,network,country
198026,1.578656e+09,126.45433,24,7665.958333,29200.0,61356.0,445.0,False,False,False,...,eth:ethertype:ip:tcp,1530,San Miguel de Cozumel,Quintana Roo,20.5004,-86.9427,axtel.net,Axtel,189.211.128.0/20,Mexico


In [201]:
def toosmall(s):
    if s<20:
        return np.nan
    else:
        return s
sub221 = sub22.groupby('event_id').count()
sub221['tooSmall'] = sub221['country'].map(toosmall)
sub221 = sub221.dropna()
case_eventid = sub221.index.tolist()
sub221

,ClusterName,country,packet_time,tooSmall
event_id,,,,
s2_c4_e7,215,215,215,215.0
s2_c7_e113,22,22,22,22.0
s2_c7_e117,20,20,20,20.0
s2_c7_e146,50,50,50,50.0
s2_c7_e182,40,40,40,40.0
s2_c7_e194,979,979,979,979.0
s2_c7_e201,59,59,59,59.0
s2_c7_e207,37,37,37,37.0
s2_c7_e211,33,33,33,33.0


In [202]:
sub222 = sub22[sub22.event_id.isin(case_eventid)]
sub222 = sub222.reset_index(drop=True)
sub222

,ClusterName,country,packet_time,event_id
0,"(80, eth:ethertype:ip:tcp)",Japan,1.578588e+09,s2_c4_e7
1,"(80, eth:ethertype:ip:tcp)",Austria,1.578588e+09,s2_c4_e7
2,"(80, eth:ethertype:ip:tcp)",Austria,1.578588e+09,s2_c4_e7
3,"(80, eth:ethertype:ip:tcp)",Austria,1.578588e+09,s2_c4_e7
4,"(80, eth:ethertype:ip:tcp)",Austria,1.578588e+09,s2_c4_e7
...,...,...,...,...
3029,"(445, eth:ethertype:ip:tcp)",Indonesia,1.578671e+09,s2_c7_e9015
3030,"(445, eth:ethertype:ip:tcp)",Venezuela,1.578671e+09,s2_c7_e9015
3031,"(445, eth:ethertype:ip:tcp)",India,1.578671e+09,s2_c7_e9015
3032,"(445, eth:ethertype:ip:tcp)",Pakistan,1.578671e+09,s2_c7_e9015


In [ ]:
1578586235.58362

In [203]:
max_time = sub222.groupby('event_id')['packet_time'].agg('max')
min_time = sub222.groupby('event_id')['packet_time'].agg('min')
sub223 = pd.concat([max_time,min_time],axis=1)
sub223.columns = ['max_time','min_time']
# type(max_time)
sub223

,max_time,min_time
event_id,,
s2_c4_e7,1.578588e+09,1.578588e+09
s2_c7_e113,1.578586e+09,1.578586e+09
s2_c7_e117,1.578586e+09,1.578586e+09
s2_c7_e146,1.578586e+09,1.578586e+09
s2_c7_e182,1.578586e+09,1.578586e+09
s2_c7_e194,1.578587e+09,1.578586e+09
s2_c7_e201,1.578587e+09,1.578587e+09
s2_c7_e207,1.578587e+09,1.578587e+09
s2_c7_e211,1.578587e+09,1.578587e+09


In [247]:
sub223.iloc[0,0]

1578588333.824687

In [255]:
sub223[(sub223.max_time>=1578586235) & (sub223.min_time<=1578586235)]

,max_time,min_time
event_id,,
